# Copy Number Pipeline

In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np
from collections import Counter

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import terra
from JKBio.utils import helper as h
from JKBio.google import gcp
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from JKBio.google.google_sheet import dfToSheet

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr,spearmanr

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display
import seaborn as sns
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [ ]:
workspace1="nci-mimoun-bi-org/CCLF_WES"
# refworkspace="nci-mimoun-bi-org/CCLF_WES_CN_HG38_tmp"
refworkspace="cclf-core/CCLF_WES_CN_HG38"
source1="cclf"
sample_set_id = "CCLF_WES_august"
release = sample_set_id

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
cnwm = dm.WorkspaceManager(refworkspace)

In [ ]:
samples = cnwm.get_samples()

In [ ]:
# refwm.delete_sample(cnwm.get_samples().index)

In [ ]:
# toprocess = pd.read_csv('data/CCLF_august/toprocess.tsv', sep='\t')

In [ ]:
# toprocess = [i.CCLE_Name+"-Tumor-" + i.CCLF_ID for p, i in toprocess.iterrows()]

In [ ]:
# samples = dm.WorkspaceManager("cclf-core/CCLF_WES_CN_HG38").get_samples()
# samples = samples[['participant']]

In [ ]:
# pairs = dm.WorkspaceManager("cclf-core/CCLF_WES_CN_HG38").get_pairs()

In [ ]:
# pair_sets = dm.WorkspaceManager("cclf-core/CCLF_WES_CN_HG38").get_pair_sets()

In [ ]:
# sample_sets = dm.WorkspaceManager("cclf-core/CCLF_WES_CN_HG38").get_sample_sets()

In [ ]:
#and CN
cnwm = cnwm.disable_hound()
cnwm.upload_samples(samples)
cnwm.upload_entities('pairs', pairs)
# cnwm.update_pair_set(pair_set_id=samplesetname, pair_ids=pairs.index)
sam = cnwm.get_samples()

# pair = cnwm.get_pairs()
# cnwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
# cnwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
# cnwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
# cnwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
cnwm.update_sample_set(sample_set_id='all', sample_ids=sample_sets.loc['all', 'samples'])
cnwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sample_sets.loc['all_agilent', 'samples'])
cnwm.update_sample_set(sample_set_id='all_ice', sample_ids=sample_sets.loc['all_ice', 'samples'])

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the copy number dataset:

*   **BamToUnmappedRGBams_MC** vdauwera/BamToUnmappedRGBamsSnapshot ID: 3
*   **Generate_uBAM_File_List** gkugener/ArrayOfFilesToTxtSnapshot ID: 1
*   **Realign_WES_GATK4** gatk/PreProcessingForVariantDiscovery_GATK4Snapshot ID: 7
*   **CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9
*   **Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2

This output file for download will be saved under the sample set under the combined_seg_file attribute.

There are several other tasks in this workspace. In brief:

*   **CNV_Somatic_Panel_Workflow_Agilent_XX** gatk/CNV_Somatic_Panel_WorkflowSnapshot ID: 11. This task was used in this workspace to generate the Sanger PON. In the Sanger dataset, there is a set of 40 normal cell lines samples (cell lines derived from matched normal tissue). We can use these to generate a PON to normalize to rather than using the Agilent PON we use for the other CCLE cell lines. This leads to less noisy results. HOWEVER, results using the PON from this workflow should not use the X chromosome, as the sanger normals are not exclusively female or male (it is likely a mix).
*   **SANGER_PON_CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9. Same as the CNV_sample_XX_gatk, except that is uses the Sanger based PON. Should be used only for the Sanger cell lines.
*   **Sanger_PON_Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2. Aggregates the segment files for the samples that were run using the Sanger PON based CNV workflow.

### cleaning workspaces

In [ ]:
torm = terra.deleteHeavyFiles(refworkspace)
h.parrun(['gsutil rm '+i for i in torm], cores=8)
terra.removeFromFailedWorkflows(refworkspace, dryrun=False, everythingFor=['Realign_WES_GATK4','Generate_uBAM_File_List','BamToUnmappedRGBams_MC','CGA_WES_CCLE_ICE','CGA_WES_CCLE_AGILENT'])

## On Terra

In [ ]:
# a list of Terra workflows that are in the workspace and that we will call sequentially
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [ ]:
# see dalmatian
subid = refwm.create_submission(bamtoubam, samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(ubamtofilelist,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(realign,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
# Testing out the XY PoN for CN characterization. Will test by producing an output in a different column from usual so it's easy to delete the column attribute later
# Also, need to make a split between Agilent and ICE samples..
submission_id= refwm.create_submission("CNV_sample_XY_ice",etype='sample_set',entity='all_ice', expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

In [ ]:
submission_id= refwm.create_submission("CNV_sample_XY_agilent",etype='sample_set',entity='all_agilent', expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

### copy pairs data to sample data

In [ ]:
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[~pairs['called_copy_ratio_segments_tumor'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

continuing

In [ ]:
submission_id = cnwm.create_submission("Aggregate_CN_seg_files", entity="all")
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
terra.waitForSubmission(refworkspace, submission_id)
aggregated = cnwm.get_entities('sample_set').loc['all']["combined_seg_file"]
aggregated

## On local

__We then save the workflow configurations used__

In [ ]:
terra.saveConfigs(refworkspace, 'data/'+sample_set_id+'/CNVconfig')

__delete unmapped bams generated during the process__

In [ ]:
toremove = ["readgroup_ubams",]
res = refwm.get_samples()
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
# sometimes the previous step does not work and you need to do it manually (you can run this to check it worked)
for val in samplesinset.readgroup_ubams:
    ubams = ''
    if not type(val) is list:
        continue 
    for v in val:
        ubams+=' '+v
    os.system('gsutil -m rm'+ubams)

In [ ]:
aggregated = 'gs://fc-secure-d8458bee-4f7e-4de1-b9d0-17df5cbe9477/7e944d7a-960a-4a1f-995a-55801d02fdc1/aggregate_CN_segments_wrkflw/d17c98f0-6754-490c-9b32-69c5f63314fe/call-aggregate_CN_segments/cacheCopy/all.called.seg'

In [ ]:
segments = pd.read_csv("temp/cnv_ccle.called.seg.tsv", sep='\t')
segments.to_csv("temp/cnv_ccle.called.seg", index)
# TODO: copy allelic calls as well

In [ ]:
len(set(segments.Sample))

In [ ]:
segments = segments.rename(columns={'CONTIG':'Chromosome',
'START':'Start',
'END':'End',
'Sample':"DepMap_ID",
'NUM_POINTS_COPY_RATIO':'Num_Probes',
'MEAN_LOG2_COPY_RATIO':'Segment_Mean',
'CALL':'Status'})

## post Procesing

The post processing happens in R using guillaume's functions, in brief:

- processSegments
- filterForCCLE
- interpolateGapsInSegmented
- extendEndsOfSegments
- reprioritizeData

In [ ]:
gene_mapping = pd.read_csv('data/genemapping_19Q1.csv')

In [ ]:
segments['Chromosome'] = [i[3:] for i in segments['Chromosome'].tolist()]
# reverting logtransform of GTK
segments.Segment_Mean = 2**segments.Segment_Mean
segments.Start = segments.Start.astype(int)
segments.End = segments.End.astype(int)
# setting sex genes to half of their value for it to match relative concentration of other genes.
segments.loc[segments[segments.Chromosome.isin(['X','Y'])].index,'Segment_Mean'] = segments[segments.Chromosome.isin(['X','Y'])]['Segment_Mean']/2
segments = segments.sort_values(by=['DepMap_ID','Chromosome','Start','End'])
gene_mapping = gene_mapping.sort_values(by=['Chromosome','start','end'])

In [ ]:
# TODO: check on IGV maxvalue

## Validation step

Once the files are saved, we load them back in python and do some validations, in brief:

- mean,max,var...
- to previous version: same mean,max,var...
- checkAmountOfSegments: flag any samples with a very high number of segments
- checkGeneChangeAccrossAll: flag any genes which stay at a similar value across all samples

In [ ]:
len(segments[segments.Segment_Mean>1000])

In [ ]:
from JKBio.mutations import manageGapsInSegments, toGeneMatrix, checkGeneChangeAccrossAll, checkAmountOfSegments

In [ ]:
gapmergedsegs = manageGapsInSegments(segments)
genecn = toGeneMatrix(gapmergedsegs, gene_mapping)

In [ ]:
genecn.to_csv('temp/gene_cn.csv')

In [ ]:
', '.join(checkGeneChangeAccrossAll(genecn, thresh=0.025))

In [ ]:
genecn.values.min(), genecn.values.mean(), genecn.values.max()

In [ ]:
failed = checkAmountOfSegments(segments, thresh = 2000)

# Upload to taiga

- we load the blacklisted/embargoed sample ids
- we log2 transform and create a file for each release (and one containing everything)
- we upload the files using taigapy in a corresponding taiga dataset with the corresponding description and also upload it to its virtual dataset

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

dataset_version_id = tc.update_dataset(
    dataset_id='cclf-cn-f083',
    changes_description='divide X values by half',
    upload_files=[
        {"path": 'temp/cnv_ccle.called.seg',
        "format": 'TableCSV',
        "name": 'wes_segment_cn_Xhalved'},
        {"path": 'temp/gene_cn.csv',
         'format': 'NumericMatrixCSV',
         "name": 'wes_gene_cn_Xhalved'}
    ],
    add_all_existing_files=True
)